<a href="https://colab.research.google.com/github/nagarajabhishek3/Google-colab-files/blob/main/rsi_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook: rsi_strategy.ipynb\n\nRSI + Bollinger Bands strategy implementation and simple backtest.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
path = '../data/nifty.csv'
try:
    df = pd.read_csv(path, parse_dates=True, index_col=0)
except Exception as e:
    print('load error:', e)
    df = pd.DataFrame()

# Compute indicators (requires 'ta' package, fallback manual RSI if not available)
if not df.empty:
    try:
        import ta
        df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
        bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
        df['bb_h'] = bb.bollinger_hband()
        df['bb_l'] = bb.bollinger_lband()
    except Exception as e:
        print('ta library not available, computing simple RSI manually:', e)
        # simple RSI implementation
        delta = df['Close'].diff()
        up = delta.clip(lower=0)
        down = -1 * delta.clip(upper=0)
        ma_up = up.rolling(14).mean()
        ma_down = down.rolling(14).mean()
        rs = ma_up / ma_down
        df['rsi'] = 100 - (100/(1+rs))
        df['ma'] = df['Close'].rolling(20).mean()
        df['std'] = df['Close'].rolling(20).std()
        df['bb_h'] = df['ma'] + 2*df['std']
        df['bb_l'] = df['ma'] - 2*df['std']

    # Signals
    df['signal'] = 0
    df.loc[(df['rsi'] < 30) & (df['Close'] <= df['bb_l']), 'signal'] = 1
    df.loc[(df['rsi'] > 70) | (df['Close'] >= df['bb_h']), 'signal'] = -1

    display(df[['Close','rsi','bb_h','bb_l','signal']].tail())

    # Simple backtest
    from src.backtest import SimpleBacktest
    bt = SimpleBacktest(df.reset_index())
    res = bt.run_signals('signal')
    print('Simple backtest pnl (sum of returns):', res['pnl'])

    # Equity curve (cumulative returns from trades)
    # NOTE: this is a simplistic approximation
    trades = [t for t in res['trades'] if t['type']=='sell']
    if trades:
        rets = [t['ret'] for t in trades if 'ret' in t]
        import numpy as np
        eq = (1 + pd.Series(rets)).cumprod()
        eq.plot(title='Equity curve (trade-level)')
        plt.show()

In [ ]:
# There are many ways to improve the backtest: apply transaction costs, use position sizing, handle partial exits, slippage, etc.